<a href="https://colab.research.google.com/github/NaraGuimma/mentoria_dados/blob/main/API_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRE-REQUISITOS

- ter uma conta no gmail (https://accounts.google.com/signin)
- criar um notebook em branco no Colab (https://colab.google/)

# PROBLEMA A SER RESOLVIDO

Queremos entender a varição do dolar, euro e libra esterlina ao longo do ano de 2024, e descobri os meses com maiores flutuações sejam elas positivas ou negativas.

# EXTRAÇAO

Extraindo os dados da fonte: https://apilayer.com/marketplace/exchangerates_data-api?utm_source=apilayermarketplace&utm_medium=featured

Por meio do uso de uma API (Application Programming Interface). Para saber mais sobre APIs leia o artigo: https://medium.com/@nara.guimaraes/tudo-o-que-voc%C3%AA-precisa-saber-sobre-apis-conceitos-exemplos-em-python-e-aplicabilidade-na-%C3%A1rea-de-edb8ab81f0c1

In [1]:
import requests
import pandas as pd
from google.colab import userdata

In [2]:
api_key = userdata.get('api_key')
start_date = '2024-01-01'
end_date = '2025-01-01'
base = 'BRL'
symbols = 'USD, EUR, GBP'

url = f"https://api.apilayer.com/exchangerates_data/timeseries?start_date={start_date}&end_date={end_date}&base={base}&symbols={symbols}"


In [3]:
payload = {}
headers= {
  "apikey": api_key
}

response = requests.request("GET", url, headers=headers, data = payload)

status_code = response.status_code
print(f"Status Code: {status_code}")
result = response.json()
print(result)
print(type(result))

Status Code: 200
{'success': True, 'timeseries': True, 'start_date': '2024-01-01', 'end_date': '2025-01-01', 'base': 'BRL', 'rates': {'2024-01-01': {'USD': 0.20602, 'EUR': 0.186651, 'GBP': 0.161866}, '2024-01-02': {'USD': 0.203088, 'EUR': 0.185614, 'GBP': 0.160894}, '2024-01-03': {'USD': 0.203223, 'EUR': 0.18599, 'GBP': 0.160353}, '2024-01-04': {'USD': 0.204203, 'EUR': 0.18654, 'GBP': 0.161001}, '2024-01-05': {'USD': 0.205124, 'EUR': 0.187247, 'GBP': 0.16118}, '2024-01-06': {'USD': 0.205145, 'EUR': 0.187267, 'GBP': 0.161214}, '2024-01-07': {'USD': 0.204771, 'EUR': 0.187145, 'GBP': 0.161036}, '2024-01-08': {'USD': 0.205318, 'EUR': 0.187497, 'GBP': 0.161116}, '2024-01-09': {'USD': 0.203807, 'EUR': 0.186444, 'GBP': 0.16035}, '2024-01-10': {'USD': 0.204378, 'EUR': 0.186228, 'GBP': 0.160366}, '2024-01-11': {'USD': 0.205288, 'EUR': 0.186945, 'GBP': 0.16067}, '2024-01-12': {'USD': 0.206007, 'EUR': 0.18792, 'GBP': 0.161587}, '2024-01-13': {'USD': 0.205982, 'EUR': 0.187897, 'GBP': 0.161605}, '2

**Criando uma estrutura no formato estruturado de tabela chamada dataframe**

In [4]:
df = pd.DataFrame.from_dict(result['rates'], orient='index').reset_index()
df

,index,USD,EUR,GBP
0,2024-01-01,0.206020,0.186651,0.161866
1,2024-01-02,0.203088,0.185614,0.160894
2,2024-01-03,0.203223,0.185990,0.160353
3,2024-01-04,0.204203,0.186540,0.161001
4,2024-01-05,0.205124,0.187247,0.161180
...,...,...,...,...
362,2024-12-28,0.161401,0.154768,0.128381
363,2024-12-29,0.161401,0.154762,0.128318
364,2024-12-30,0.161828,0.155519,0.128965
365,2024-12-31,0.161828,0.155519,0.128965


In [5]:
df.describe()

,USD,EUR,GBP
count,367.000000,367.000000,367.000000
mean,0.186214,0.172049,0.145805
std,0.012544,0.010946,0.010808
min,0.148502,0.142807,0.118395
25%,0.177072,0.162743,0.136527
50%,0.184003,0.169052,0.142869
75%,0.198766,0.183479,0.157168
max,0.206020,0.188503,0.161866


In [6]:
df.nunique()

,0
index,367
USD,329
EUR,346
GBP,358


In [7]:
df.isnull().sum().sum() # nao tem valores nulos

0

# CARREGAMENTO NO DRIVE

**Salvando em csv no drive, o qual funcionará como nosso data lake**

---




In [8]:
from google.colab import auth
from google.auth import default
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import drive

In [9]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [10]:
df.to_csv('/content/drive/My Drive/Mentoria_Dados/taxa_de_cambio.csv', index=False)

**Salvando como planilha do google sheets no drive, o qual funcionará como nosso data lake**


Para saber mais: https://medium.com/@nara.guimaraes/como-salvar-um-dataframe-do-pandas-no-google-sheets-usando-google-colab-17d4fb04ec45

In [11]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [12]:
# Cria uma nova planilha do no Google Sheet
spreadsheet_name = "taxa_de_cambio_sheets"
spreadsheet = gc.create(spreadsheet_name)

In [14]:
sheet = gc.open(spreadsheet_name).sheet1

# Upload do Dataframe para a planilha criada
set_with_dataframe(sheet, df)

print(f"Dados salvos com sucesso na planilha: {spreadsheet_name}")

Dados salvos com sucesso na planilha: taxa_de_cambio_sheets


**Verificando o arquivo criado**

In [15]:
import pandas as pd

# Obter todos os dados da planilha
rows = sheet.get_all_values()

# Converter para DataFrame
headers = rows[0]  # Cabeçalhos
values = rows[1:]  # Dados

df_novo = pd.DataFrame(values, columns=headers)
print(df_novo)

          index       USD       EUR       GBP
0    2024-01-01   0.20602  0.186651  0.161866
1    2024-01-02  0.203088  0.185614  0.160894
2    2024-01-03  0.203223   0.18599  0.160353
3    2024-01-04  0.204203   0.18654  0.161001
4    2024-01-05  0.205124  0.187247   0.16118
..          ...       ...       ...       ...
362  2024-12-28  0.161401  0.154768  0.128381
363  2024-12-29  0.161401  0.154762  0.128318
364  2024-12-30  0.161828  0.155519  0.128965
365  2024-12-31  0.161828  0.155519  0.128965
366  2025-01-01  0.158692  0.153306  0.126767

[367 rows x 4 columns]
